# タスク 2: はじめての AI エージェントを構築する

このノートブックでは、Strands Agents フレームワークを使用して AI エージェントを構築する方法を学びます。まず会話エージェントを作成し、次にツールを追加してその機能を強化します。最後に、Web から料理情報を検索できる実用的なレシピアシスタントを構築します。エージェントの実行には Amazon Bedrock と Amazon Nova Lite モデルを使用します。

AI エージェントは、従来の LLM と異なり、自律的にアクションを実行し、ツールを使用し、目標に向かって作業することができます。質問に応答するだけでなく、実際にタスクを実行することができます。

#### シナリオ
あなたは、日常的なタスクの自動化に AI エージェントがどのように役立つかを探求したいと考えている成長中のテクノロジー企業 AnyCompany で働いています。あなたのチームは、これらのエージェントがリサーチやカスタマーサポートなどのタスクにどのように役立つかについて学ぶことに関心があります。

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **詳細情報:** このノートブックを進める際、Amazon Q Developer を使用して Python コードとロジックを理解することができます。Amazon Q はコードスニペットを説明し、改善を提案し、Python プログラミングの概念を学ぶのに役立ちます。

**JupyterLab で Amazon Q Developer にアクセスするには**

<!-- ![JupyterLab での Amazon Q の場所](./images/amazon-q-location.png) -->
<img src="images/amazon-q-location.png" width=50% height=20% />

*画像の説明: 上記のスクリーンショットは、左サイドバーに Amazon Q パネルが表示された JupyterLab インターフェースを示しています。Amazon Q アイコンとチャットインターフェースが強調表示され、受講者が Amazon Q Developer 機能にアクセスできる場所を示しています。*

1. JupyterLab インターフェースの左サイドバーにある **Amazon Q** パネルを探します（上の画像のとおり）
2. 接続したら、以下のことができます
   - コードブロックセルを選択し、右クリックして **Generative AI** > **Explain code** を選択
   - Amazon Q チャットで Python の構文、関数、またはプログラミングの概念について質問
   - コードの提案や改善点を依頼
   - エラーのデバッグに関するヘルプを取得

## タスク 2.1: 環境セットアップ

このタスクでは、AI エージェントの構築を始めるために必要なパッケージをインストールして環境をセットアップします。

In [ ]:
# Strands Agents フレームワークとツールをインストール
%pip install strands-agents strands-agents-tools

## タスク 2.2: 最初の AI エージェントを作成

このタスクでは、会話ができる AI エージェントを作成します。このエージェントは Amazon Bedrock と Amazon Nova Lite モデルを使用してメッセージを理解し応答します。システムプロンプトはエージェントの望ましい動作を定義します。

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

from strands import Agent

# はじめての AI エージェントを作成
agent = Agent(callback_handler=None,
    model="amazon.nova-lite-v1:0",
    system_prompt="あなたは簡潔な回答を提供する親切なアシスタントです。"
)

次に、エージェントにメッセージを送信してテストします。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注意:** Strands フレームワークはデフォルトで Amazon Bedrock と Amazon Nova Lite モデルを使用します。このモデルは会話的なインタラクションをサポートし、ツールで機能を強化できます。

In [ ]:
# エージェントにメッセージを送信
response = agent("こんにちは！ジョークを言ってください。")
print(response)

## タスク 2.3: エージェントにツールを追加

このタスクでは、エージェントにツールを提供して、チャット以外のことができるようにします。

- Strands Agents SDK が提供する計算機ツールを追加します。

- @tool デコレータを使用して天気ツールを作成します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注意:** 天気ツールはプレースホルダーの例で、常に「晴れ」を返します。

In [ ]:
from strands import Agent, tool
from strands_tools import calculator

# 天気ツールを作成
@tool
def weather():
    """Get current weather information"""
    return "晴れ、摂氏0度"

# Create an agent with tools
agent_with_tools = Agent(callback_handler=None,
    model="amazon.nova-lite-v1:0",
    tools=[calculator, weather],
    system_prompt="あなたは役に立つアシスタントです。数学の計算をしたり、天気ツールを使って天気を知らせたりできます。"
)

# 両方のツールを1つのクエリでテスト
# 注意: このクエリは両方のツールを必要とします - 摂氏の温度を取得する天気ツールと、
# 摂氏から華氏に変換する計算機ツール。
# エージェントは自動的にどのツールを使用し、どの順序で使用するかを判断します。
response = agent_with_tools("シアトルの天気は華氏何度ですか?")
print(response)

次に、エージェントが質問を分析し、計算機ツールのみを使用することを決定する方法を探ります。

In [ ]:
# 計算機ツールのみが必要な数学の質問をテストする
# エージェントは質問を分析し、計算機ツールのみを使用することを決定します
math_query = "25 * 4 + 18 はいくらですか?"
print("=== Agent Chooses Calculator Tool ===")
print(f"Query: {math_query}")
response = agent_with_tools(math_query)
print(f"Response: {response}")

### ツールの直接呼び出し

エージェントの会話を介さずに、ツールを直接呼び出すこともできます。これは特定のツール機能をテストしたい場合や使用したい場合に便利です。

**ツールの直接呼び出しの仕様:**
- 特定のツールを直接呼び出すには `agent.tool.tool_name()` を使用
- 必要なパラメータを関数の引数として渡す
- エージェントの自然言語処理とツール選択のロジックをバイパス
- ツール機能へのプログラム的なアクセスに便利

In [ ]:
# 計算機ツールを直接呼び出し
# 注意: ツールの直接呼び出しはエージェントの会話フローをバイパスします。
# agent_with_tools.tool.calculator() を使用して計算機ツールを直接呼び出し、
# エージェントにツールの選択を任せません。これは特定のツールをテストする場合や、
# 必要なツール機能を正確に知っている場合に便利です。
result = agent_with_tools.tool.calculator(expression="2 + 3 * 4")
print(f"Calculator result: {result}")

## タスク 2.4: ロギングの設定

このタスクでは、エージェントの裏側での動作を理解するためにロギングを設定します。これにより、エージェントがリクエストをどのように処理し、ツールを使用するかを理解するのに役立ちます。

Strands フレームワークは Python の標準ロギングモジュールを使用して、エージェントの操作の可視性を提供します。異なるログレベルを設定して、エージェントの動作に関する詳細情報を多くまたは少なく取得できます。

In [ ]:
import logging
from strands import Agent
import os

# エージェントの動作を理解するための詳細なロギングを有効化
logging.getLogger("strands").setLevel(logging.INFO)

# コンソールとファイルの両方に書き込むようにロギングを設定
logging.basicConfig(
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    level=logging.INFO,
    handlers=[
        logging.StreamHandler()  # Console output
    ]
)

# ロガーを作成
logger = logging.getLogger("agent_activity")

# ロギングを有効にしてエージェントを作成
logger.info("Creating new agent with Nova Lite model")
logged_agent = Agent(callback_handler=None,model="amazon.nova-lite-v1:0")

logger.info("Sending message to agent: 'こんにちは お元気ですか？'")
response = logged_agent("こんにちは お元気ですか？")
print(response)

## タスク 2.5: モデル設定の探索

このタスクでは、エージェントの異なるモデルと設定を構成する方法を学びます。使用する AI モデルを指定し、temperature などのパラメータでその動作を調整できます。temperature は応答の創造的や一貫性を制御します。

In [ ]:
from strands import Agent
from strands.models import BedrockModel

# カスタムモデル設定を作成
custom_model = BedrockModel(
    model_id="amazon.nova-lite-v1:0",
    temperature=0.3  # Lower temperature = more consistent responses
)

# カスタムモデルでエージェントを作成
custom_agent = Agent(callback_handler=None,model=custom_model)
print("Agent created successfully!")

## タスク 2.6: レシピアシスタントエージェントの構築

このタスクでは、料理を支援できるより実用的なエージェントを作成します。このレシピアシスタントは、レシピや料理情報をウェブで検索でき、エージェントが実際のシナリオでどのように役立つかを示します。

まず、レシピエージェントが必要とするウェブ検索パッケージをインストールします。

In [ ]:
%pip install ddgs

レシピエージェントがオンラインで料理情報やレシピを検索できるウェブ検索ツールを作成します。

In [ ]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# ロギングを設定
logging.getLogger("strands").setLevel(logging.INFO)

# ウェブ検索ツールを作成
@tool
def websearch(keywords: str, max_results: int = 3) -> str:
    """Search the web for information.
    Args:
        keywords (str): What to search for
        max_results (int): How many results to return
    Returns:
        Search results as text
    """
    try:
        results = DDGS().text(keywords, max_results=max_results)
        return results if results else "No results found."
    except Exception as e:
        return f"Search error: {e}"

print("Web search tool created successfully!")

次に、ウェブ検索ツールを使用して料理に関する質問に答えるレシピアシスタントエージェントを作成します。

In [ ]:

# レシピアシスタントエージェントを作成
recipe_agent = Agent(callback_handler=None,
    model="amazon.nova-lite-v1:0",
    system_prompt="""あなたは、役立つ料理アシスタント「RecipeBot」です。
ユーザーがレシピを見つけたり、料理に関する質問に答えたりできるようサポートしましょう。
ウェブ検索ツールを使用して、レシピや料理情報を検索しましょう。""",
    tools=[websearch]
)

print("Recipe assistant agent created successfully!")

料理のヘルプを求めてレシピアシスタントをテストします。ウェブ検索ツールを使用して最新の情報を見つける様子を観察してください。

In [ ]:
# レシピアシスタントをテストする
response = recipe_agent("鶏肉とブロッコリーを使った簡単なレシピを提案して下さい。")
print(response)

ウェブを検索して料理の質問に答えることができる実用的な AI エージェントの作成に成功しました。これは、エージェントが実際のタスクにどのように役立つかを示しています。

これで、ツールを使用してタスクを実行できる AI エージェントを構築する方法を提供する Strands Agents フレームワークを試してみました。このフレームワークを使用して、エージェントが従来の LLM と異なり、ツールを積極的に使用してタスクを実行できることを学びました。

### 自分で試してみましょう
- システムプロンプトを変更して、異なるユースケース用のエージェントを作成する
- チームが必要とする特定のタスク用のカスタムツールを作成する
- さまざまなモデル設定を試して、エージェントの動作にどのような影響があるか理解する

### まとめ

このノートブックを完了しました。ラボの次のパートに進むには、以下を実行してください。

- このノートブックファイルを閉じて、**結論**に進んでください。